# Blatt 4

Hi, Hallo, herzlichst Willkommen! Hier ist das [Blatt](DBS_SS23_Uebungsblatt-04.pdf). Diesmal gibt es mehrere Aufgaben zu SQL, diese werden hier anstatt bei OneNote bearbeitet, weil ich gestern 10 Minuten nach meinen Lösungen für Blatt 3 bei PyCharm gesucht habe.

    ## Aufgabe 1
Damit man Aufgabe 1 versteht, muss ich mir erstmal ein paar Definitionen machen, wenn ich später lernen will, Atzmüller erklärt das alles kacke.
#### Funktionale Abhängigkeiten
Eine funktionale Abhängigkeit (FA) ist eine Abhängigkeit zwischen Spalten (Attributen) einer oder mehrerer Tabellen (Relationen).

Eine funktionale Abhängigkeit zwischen zwei Attributmengen wird definiert als eine Beziehung, bei der der Wert einer oder mehrerer Attribute in der einen Menge den Wert anderer Attribute in der gleichen oder einer anderen Menge eindeutig bestimmt. Mit anderen Worten, wenn eine funktionale Abhängigkeit besteht, kann der Wert einer bestimmten Attributmengen-Kombination den Wert einer anderen Attributmengen-Kombination vorhersagen.

Eine funktionale Abhängigkeit wird in der Form $X \rightarrow Y$ dargestellt, wobei $X$ und $Y$ Attributmengen sind. $X$ wird als linke Seite (Determinante) bezeichnet, während $Y$ als rechte Seite (Abhängige) bezeichnet wird. Diese Notation bedeutet, dass der Wert von $Y$ eindeutig durch den Wert von $X$ bestimmt wird.

Um zu bestimmen, ob es sich bei den funktionalen Abhängigkeiten um Superschlüssel oder Kandidatenschlüssel handelt, müssen wir die Eindeutigkeit der Tupel überprüfen.

Die gegebene Relation ist $R(A, B, C, D, E)$ mit den funktionalen Abhängigkeiten:

$$
A   \rightarrow B \newline
AB  \rightarrow CDE\\
C   \rightarrow DE\\
E   \rightarrow DB\\
DE  \rightarrow AC
$$

## Aufgabe 2
In Aufgabe 2 sollen wir die **UniDB** optimieren, indem wir Informationen in **Relationstabellen** auslagern und diese entsprechend beim Löschen der zugehörigen Relationen **kaskadieren** oder **nullifizieren** lassen.

#### 1. Rooms
Es ist unmöglich eine Spalte auf 2 ForeignKeys zu erweitern. Wir können der Spalte zwar zwei Constraints anfügen, aber dann werden sich diese gegenseitig Fehler aufwerfen. Wir erstellen für die beiden ForeignKeys aus Professor und Assistant die zugehörigen Spalten; außerdem definieren wir uns die Spalten RoomNo und BuildNo, die jeweils die Raumnummer beinhaltet, sowie wahrscheinlich zukünftig eine Gebäudenummer in einer anderen Tabelle.

Zu guter Letzt definieren wir uns zwei Checks, die sichergehen, dass genau eine der Spalten PerNoProf oder PerNoAssi einen Wert hat; also eine XOR-Bedingung:

| PerNoProf | PerNoAssi | Akzeptiert? |
|:---------:|:---------:|:-----------:|
|   Nein    |   Nein    |    Nein     |
|   Nein    |    Ja     |     Ja      |
|    Ja     |   Nein    |     Ja      |
|    Ja     |    Ja     |    Nein     |

Der SQL-Befehl sieht folgendermaßen aus:

```sql
create table Room
(
    BuildNo int default 0,
    RoomNo int not null,
    PerNoProf int null,
    PerNoAssi int null,

    foreign key (PerNoProf) references Professor(PerNo) on delete set null,
    foreign key (PerNoAssi) references Assistant(PerNo) on delete set null,

    check ( (Room.PerNoAssi is not null and Room.PerNoProf is null) or (Room.PerNoProf is not null and Room.PerNoAssi is null))
);
```

Im Anschluss daran können wir die Beziehungen aus Professor auskoppeln und schließlich die Spalten löschen:

```sql
insert into Room (Per_No_Prof, Room_No) (select PerNo, Room from Professor);
alter table Professor drop column Room;
commit;
```

#### 2. teaches

Wir erstellen eine Tabelle teach, die die Relationen zwischen Professor und Kurs speichert. Beim Löschen eines Tupels aus Professor kaskadiert sich die Löschen-Aktion zu teach und löscht die korrespondierenden Einträge.

## Aufgabe 4

#### 1. Was ist ein Full Outer Join?

Ein Full Outer Join ist eine Art von Join-Operation in einer Datenbank, die Datensätze aus zwei Tabellen basierend auf einer gemeinsamen Spalte oder Bedingung kombiniert. Im Gegensatz zu anderen Joins wie dem Inner Join oder dem Left Outer Join, gibt der Full Outer Join alle Zeilen aus beiden Tabellen zurück, auch wenn es keine Übereinstimmungen zwischen den Tabellen gibt.

Die Funktionsweise eines Full Outer Joins lässt sich am besten anhand eines Beispiels erklären. Angenommen, wir haben zwei Tabellen: "Tabelle A" und "Tabelle B". Beide Tabellen enthalten eine Spalte namens "ID", die als gemeinsame Spalte fungiert.

Der Full Outer Join kombiniert die Datensätze beider Tabellen basierend auf der ID-Spalte. Dabei gibt es drei mögliche Szenarien:

1. Wenn eine ID in beiden Tabellen vorhanden ist, werden die entsprechenden Zeilen aus beiden Tabellen kombiniert.
2. Wenn eine ID nur in Tabelle A vorhanden ist, wird die Zeile aus Tabelle A beibehalten und die Werte in den Spalten von Tabelle B werden mit NULL-Werten aufgefüllt.
3. Wenn eine ID nur in Tabelle B vorhanden ist, wird die Zeile aus Tabelle B beibehalten und die Werte in den Spalten von Tabelle A werden mit NULL-Werten aufgefüllt.

Das Ergebnis des Full Outer Joins ist eine kombinierte Tabelle mit allen Zeilen aus beiden Tabellen, wobei NULL-Werte verwendet werden, wenn keine Übereinstimmung in der gemeinsamen Spalte gefunden wird.

#### 2. Wie kann man einen Full Outer Join in MariaDB simulieren?

Man kann einen Right-Join und einen Left-Join machen. Danach verbindet man beide per Union.

```sql
use uniDB;
select * from Assistant left join Professor P on P.PerNo = Assistant.Boss
union
select * from Assistant right join Professor P2 on P2.PerNo = Assistant.Boss;
```

#### 3. FOJ für Student und Kurs:

Man macht das wie folgt: Man holt sich alle Spalten aus dem linken Join der Student-Tabelle mit der attend-Tabelle, dadurch bekommen wir eine Tabelle wo auf jeden Fall jeder Student vertreten ist, aber nicht jeder Student ist in attend vertreten. Mit dieser Tabelle machen wir einen linken Join zu Course, um uns die Informationen zu den Kursen zu holen. Auf der anderen Seite machen wir einen rechten Join von Student auf attend, das gibt uns alle Spalten aus attend, womöglich nicht alle aus Student. Dann machen wir einen rechten Join auf Course, d.h. es sind mindestens alle Kurse in der Tabelle vertreten, aber nicht mehr alle Studenten.
```sql
select * from Student left join attend a on Student.StuNo = a.StuNo left join Course C on a.CouNo = C.CouNo
union
select * from Student right join attend a on Student.StuNo = a.StuNo right join Course C on a.CouNo = C.CouNo;
```

insert into teach (TaughtBy, CouNo) (SELECT TaugthBy, CouNo From Professor RIGHT JOIN Course C on Professor.PerNo = C.TaugthBy)